<a href="https://colab.research.google.com/github/tawseeq1/GPT_for_shakespeare/blob/main/gpt_ak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-23 13:48:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2024-10-23 13:48:23 (12.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
text[:122]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rath'

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i : ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
print(encode("taw  seeq"))
print(decode([58, 39, 61, 57, 43, 43, 55]))

[58, 39, 61, 1, 1, 57, 43, 43, 55]
tawseeq


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [ ]:
n = int(0.9 * len(text))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"when context is {context}, target is {target}")

when context is tensor([18]), target is 47
when context is tensor([18, 47]), target is 56
when context is tensor([18, 47, 56]), target is 57
when context is tensor([18, 47, 56, 57]), target is 58
when context is tensor([18, 47, 56, 57, 58]), target is 1
when context is tensor([18, 47, 56, 57, 58,  1]), target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i+1:i+1 + block_size] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('-------inputs--------')
print(xb.shape)
print(xb)
print('-------targets-------')
print(yb.shape)
print(yb)

print('----')

-------inputs--------
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
-------targets-------
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [ ]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is [52] the target: 58
when input is [52, 58] the target: 1
when input is [52, 58, 1] the target: 58
when input is [52, 58, 1, 58] the target: 46
when input is [52, 58, 1, 58, 46] the target: 39
when input is [52, 58, 1, 58, 46, 39] the t

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as ff
torch.manual_seed(42)

In [ ]:
class BigramModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = ff.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
      for _ in range(max_new_tokens):
          logits, loss = self(idx)
          logits = logits[:, -1, :]
          probs = ff.softmax(logits, dim=-1)
          idx_next = torch.multinomial(probs, num_samples=1)
          idx = torch.cat((idx, idx_next), dim=1)
      return idx


m = BigramModel(vocab_size)
out , loss= m(xb, yb)
print(out.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))

torch.Size([256, 65])
tensor(4.9024, grad_fn=<NllLossBackward0>)

,yPTD pNeMcQPviBRPXhR
t3;foVM&$xpA,tR
H:$YJXN!QD&nkqxFnm&l 'fKLa$nyD KCHoh,BLMMAhjJx'J:cWeJ3MVVM3YWc


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.420147657394409


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 500)[0].tolist()))


Sheandok ake at fe anded d
ORSmy arermbblay finee ist bldingorded stare rewha chat me me botigeve, t ma beriu pt athared MBE iears hanowh in arol.
I on.
AGSat If as avee
ICENar ENRWI th; here,

Was by our semad ves thrrcear y s hawsped
ETon ur.
Anth h whlarswinonowedly keleainghe mbe;
Go the asiomulldite ck metoustharoupakid s peckinthagamathe; bure is ince sstr l m d, nvis thesthod blindilt.
D:
Wheedikiroomy berere? hawfousord

HUREOFIOn ive tent s.
Gound? IUTof!
Aneel prerelerrey ha n hityon'T


### Self Attention

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, : t + 1]
    xbow[b, t] = torch.mean(xprev,0)


In [ ]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [ ]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
# x.shape
xbow2 = wei @ x #(4' * 8*8) @ (4 * 8 * 2)
xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
# softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = ff.softmax(wei, dim=-1)
xbow3 = wei @ x
# torch.allclose(xbow, xbow3)
xbow3[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)
wei = wei/ (head_size ** 0.5)
tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = ff.softmax(wei, dim=-1)

v = value(x)
xbow3 = wei @ v
# torch.allclose(xbow, xbow3)
xbow3.shape

torch.Size([4, 8, 16])